In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import tqdm
import torch.nn.functional as F
import tqdm
import impute
import PMLE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import datetime

/Users/remycanario/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)
Using TensorFlow backend.


# Data Preprocessing

In [2]:
df = pd.read_csv('testing_df.csv',index_col=0)
df.drop('TransactionID',axis=1,inplace=True)


/Users/remycanario/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1. Prune Columns

In [3]:
dropped_cols = []

#drop columns that have excessive NA 
for col in df.columns:
    if df[col].isnull().sum()>0.95*df.shape[0]:
        df.drop(col,axis=1,inplace=True)
        dropped_cols.append(col)
        
#drop columns that have only one value
for col in df.min()[df.min()==df.max()].index:
    df.drop(col,axis=1,inplace=True)
    dropped_cols.append(col)

## 2. Feature Engineering

In [4]:
X = df.drop('isFraud',axis=1)
y = df['isFraud']

#Create feature for number of NAs per column
row_nas = []
for i in range(X.shape[0]):
    row_nas.append(X.iloc[i,:].isna().sum())
X['NAs']= row_nas

#Create features of characteristics of transaction amounts
X['amtby5']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[0]) % 5==0 else 0)
X['centsby5']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1]) % 5==0 else 0)
X['threedec']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1]) > 99 else 0)
X['centsnot0']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1][:2])!=0 else 0)
X['centsnot95']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1][:2])!=95 else 0)
X['digits']= X.TransactionAmt.apply(lambda x: str(x).split('.')[0][-1])
X['places']= X.TransactionAmt.apply(lambda x: str(len(str(x).split('.')[0])))


#Prepare card columns for one hot encoding 
X.loc[:,'card1':'card5'].astype(str)

#Convert TransactionID into datetime and derive features
X['Datetime'] = X.TransactionDT.apply(lambda x:datetime.datetime(2018,1,1)+datetime.timedelta(seconds=x))
X['Weekday']= X.Datetime.apply(lambda x: x.weekday())
X['Month']= X.Datetime.apply(lambda x: x.month)
X['Hour']= X.Datetime.apply(lambda x: x.hour)
X['Day']=(X.TransactionDT - 86400)//(60*60*24)
X['Week']= X.Day//7
X['Minute']= X.Datetime.apply(lambda x: x.minute)
X.drop('Datetime',axis=1,inplace=True)

X = pd.get_dummies(X,drop_first=True,dummy_na=True)

## 3. Train Test Split

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.2,random_state=0)

## 4. Fill Missing Values with Multiple Imputation

In [6]:
#Establish floors and ceilings for column values
column_ranges = {}
for col in X_train.columns:
    column_ranges[col]=(X_train[col].min(),X_train[col].max())

#Impute missing values using multiple imputation
X_train, train_problems = impute.impute_missing_values(X_train,col_floor_ceiling_dict=column_ranges)
X_val, val_problems = impute.impute_missing_values(X_val,col_floor_ceiling_dict=column_ranges)

print('X_train NAs = ', X_train.isna().sum().sum())
print('X_val NAs = ', X_train.isna().sum().sum())

  0%|          | 0/314 [00:00<?, ?it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  0%|          | 1/314 [00:01<05:43,  1.10s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  1%|          | 2/314 [00:02<05:41,  1.09s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

  7%|▋         | 22/314 [00:21<04:33,  1.07it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  7%|▋         | 23/314 [00:22<04:29,  1.08it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  8%|▊         | 24/314 [00:23<04:30,  1.07it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 14%|█▍        | 44/314 [00:44<04:47,  1.06s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 14%|█▍        | 45/314 [00:45<04:35,  1.02s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 15%|█▍        | 46/314 [00:46<05:07,  1.15s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 21%|██        | 66/314 [01:09<04:50,  1.17s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 21%|██▏       | 67/314 [01:10<04:51,  1.18s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 22%|██▏       | 68/314 [01:12<04:38,  1.13s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 28%|██▊       | 88/314 [01:35<04:25,  1.17s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 28%|██▊       | 89/314 [01:37<04:15,  1.14s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 29%|██▊       | 90/314 [01:38<04:19,  1.16s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

...

 75%|███████▌  | 236/314 [03:01<02:02,  1.57s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 75%|███████▌  | 237/314 [03:02<01:48,  1.41s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 76%|███████▌  | 238/314 [03:04<01:49,  1.44s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 82%|████████▏ | 258/314 [03:33<01:33,  1.67s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 82%|████████▏ | 259/314 [03:34<01:23,  1.52s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 83%|████████▎ | 260/314 [03:36<01:21,  1.51s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 89%|████████▉ | 280/314 [04:05<00:47,  1.39s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 89%|████████▉ | 281/314 [04:07<00:50,  1.53s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 90%|████████▉ | 282/314 [04:08<00:44,  1.40s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 96%|█████████▌| 302/314 [04:41<00:20,  1.69s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 96%|█████████▋| 303/314 [04:42<00:18,  1.67s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 97%|█████████▋| 304/314 [04:43<00:15,  1.55s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

X_train NAs =  0
X_val NAs =  0


## 5. Standardize

In [7]:
#Save meta data
columns = X_train.columns
normal_inds = y_train[y_train==0].reset_index().index

#Standardize dfs
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_val_sc = sc.fit_transform(X_val)

# Autoencoder Anomaly Detection
### With undercomplete, regularized and denoising autoencoders using pytorch

## Data Processing

In [8]:
n_features = X_train_sc.shape[1]

#Convert non-fraud rows to pytorch tensor
normal_train = X_train_sc[normal_inds,:]
normal_torch = torch.from_numpy(normal_train).type(torch.FloatTensor)

#Convert train and val sets to pytorch tensor
train_torch = torch.from_numpy(X_train_sc).type(torch.FloatTensor)
val_torch = torch.from_numpy(X_val_sc).type(torch.FloatTensor)

## Create Pytorch objects

In [11]:
class AutoEncoder(nn.Module):
    
    def __init__(self,n_features,hidden_nodes,dropout=None,VAE=False):
        
        '''PARAMATERS
           n_features: number of X variables in dataset
           hidden_nodes: number of nodes in hidden layer
           dropout: fraction of nodes to dropout (0 < dropout <1)'''
        
        super(AutoEncoder, self).__init__()
        self.n_features=n_features
        self.n_hidden = hidden_nodes
        self.encoder = nn.Linear(n_features,hidden_nodes)
        self.decoder = nn.Linear(hidden_nodes,n_features)
        self.output_layer = nn.Linear(n_features,n_features)
        self.dropout = dropout
        
        
    def forward(self,x):
        if self.dropout!=None:
            x = F.relu(F.dropout(self.encoder(x)))
        else:
            x = F.relu(self.encoder(x))
        self.hidden_layer=x
        x = F.relu(self.decoder(x))
        x = self.output_layer(x)
        return x
    

In [10]:
def train_autoencoder(model, dataset, loss_func, optimizer, validation_tensor=None, epochs=100, batch_size=1024, 
                      noise_factor=None, random_seed=None, MSE_stopping_threshold=0):
    '''Parameters
       model: instantiated autoencoder
       dataset: numpy array of X variables
       loss_func: instantiated loss function
       optimizer: instantiated optimizer
       epochs: number of epochs
       batch_size: batch_size
       noise_factor: magnitude of noise added to data
         for a denoising autoencoder (0 < noise_factor <=1)
       random_seed: random_seed
       stopping_MSE_threshold: MSE value after which autoencoder stops training'''
    
    if random_seed!=None:
        torch.manual_seed(random_seed)
    train_loader = torch.utils.data.DataLoader(dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)
    
    if type(validation_tensor)==torch.Tensor:
        val = True
        val_numpy = validation_tensor.detach().numpy()
        
    readout_batch_interval = .25*(dataset.shape[0]/batch_size)//1
    
    for epoch in range(epochs):
        counter = 0
        print('\n\033[1mEpoch {}\033[0m\n'.format(epoch+1))
        for batch in train_loader:
            
            if noise_factor!=None:
                batch = batch + noise_factor * torch.randn(*batch.shape)
            batch = torch.autograd.Variable(batch)
            optimizer.zero_grad()
            recon = model(batch)
            loss = loss_func(recon, batch)
            
            if (counter%readout_batch_interval==0):
                print('Batch {} Loss: {:.4f}'.format(counter, float(loss)))
            loss.backward()
            optimizer.step()
            counter+=1
        if epoch==0:
            epoch_loss = loss_func(model(dataset), dataset)
            print('\nEPOCH {} LOSS: {:.4f}'.format(epoch+1, float(epoch_loss)))
        else:
            old_epoch_loss = epoch_loss
            epoch_loss = loss_func(model(dataset), dataset)
            print('\nEPOCH {} LOSS: {:.4f}'.format(epoch+1, float(epoch_loss)))
            print('\nChange: {:.4f}%'.format(float((epoch_loss-old_epoch_loss)/old_epoch_loss)))
        if val==True:
            val_output = model(validation_tensor).detach().numpy()
            reconstruction_error = np.sqrt(np.power(val_output - val_numpy, 2)).sum(axis=1)
            reconstruction_error = reconstruction_error/(reconstruction_error.max()-reconstruction_error.min())
            print('\nReconstruction error/fraud corr: {:.4f}'.format(np.corrcoef(y_val,reconstruction_error)[0][1]))
        if epoch_loss<=MSE_stopping_threshold:
            break

## Train Autoencoders

### Autoencoder I: Dropout

#### Instantiate encoder and loss function

In [11]:
ae1 = AutoEncoder(n_features,int(n_features*1.5//1),dropout=0.3)
loss_func = nn.MSELoss()

NameError: name 'AutoEncoder' is not defined

#### Train autoencoder

In [74]:
#Start with high learning rate
optimizer = torch.optim.SGD(ae1.parameters(), lr=0.2, momentum=0.9,nesterov=True)
train_autoencoder(model=ae1,
                   dataset=normal_torch,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=256,
                   epochs=150,
                   MSE_stopping_threshold=0.19)


Epoch 1

Batch 0 Loss: 0.4051
Batch 12 Loss: 0.4481
Batch 24 Loss: 0.6679
Batch 36 Loss: 0.4855

EPOCH 1 LOSS: 0.4714

Epoch 2

Batch 0 Loss: 0.4584
Batch 12 Loss: 0.4880
Batch 24 Loss: 0.4910
Batch 36 Loss: 0.4610

EPOCH 2 LOSS: 0.4663

Epoch 3

Batch 0 Loss: 0.4417
Batch 12 Loss: 0.4419
Batch 24 Loss: 0.4506
Batch 36 Loss: 0.4612

EPOCH 3 LOSS: 0.4589

Epoch 4

Batch 0 Loss: 0.6206
Batch 12 Loss: 0.4401
Batch 24 Loss: 0.4433
Batch 36 Loss: 0.4580

EPOCH 4 LOSS: 0.4528

Epoch 5

Batch 0 Loss: 0.4068
Batch 12 Loss: 0.4385
Batch 24 Loss: 0.4638
Batch 36 Loss: 0.4842

EPOCH 5 LOSS: 0.4492

Epoch 6

Batch 0 Loss: 0.5066
Batch 12 Loss: 0.4785
Batch 24 Loss: 0.4814
Batch 36 Loss: 0.4902

EPOCH 6 LOSS: 0.4489

Epoch 7

Batch 0 Loss: 0.4036
Batch 12 Loss: 0.4772
Batch 24 Loss: 0.4104
Batch 36 Loss: 0.5058

EPOCH 7 LOSS: 0.4437

Epoch 8

Batch 0 Loss: 0.4723
Batch 12 Loss: 0.4531
Batch 24 Loss: 0.4273
Batch 36 Loss: 0.3422

EPOCH 8 LOSS: 0.4384

Epoch 9

Batch 0 Loss: 0.4127
Batch 12 Loss: 0.

Batch 36 Loss: 0.3992

EPOCH 64 LOSS: 0.3396

Epoch 65

Batch 0 Loss: 0.3297
Batch 12 Loss: 0.3674
Batch 24 Loss: 0.3337
Batch 36 Loss: 0.3947

EPOCH 65 LOSS: 0.3405

Epoch 66

Batch 0 Loss: 0.3760
Batch 12 Loss: 0.3122
Batch 24 Loss: 0.3493
Batch 36 Loss: 0.3112

EPOCH 66 LOSS: 0.3388

Epoch 67

Batch 0 Loss: 0.3494
Batch 12 Loss: 0.3819
Batch 24 Loss: 0.3060
Batch 36 Loss: 0.3296

EPOCH 67 LOSS: 0.3352

Epoch 68

Batch 0 Loss: 0.3725
Batch 12 Loss: 0.2629
Batch 24 Loss: 0.3175
Batch 36 Loss: 0.3365

EPOCH 68 LOSS: 0.3363

Epoch 69

Batch 0 Loss: 0.3071
Batch 12 Loss: 0.3748
Batch 24 Loss: 0.3119
Batch 36 Loss: 0.2831

EPOCH 69 LOSS: 0.3341

Epoch 70

Batch 0 Loss: 0.3957
Batch 12 Loss: 0.3260
Batch 24 Loss: 0.3608
Batch 36 Loss: 0.3100

EPOCH 70 LOSS: 0.3313

Epoch 71

Batch 0 Loss: 0.3195
Batch 12 Loss: 0.3911
Batch 24 Loss: 0.3532
Batch 36 Loss: 0.3214

EPOCH 71 LOSS: 0.3324

Epoch 72

Batch 0 Loss: 0.4321
Batch 12 Loss: 0.3731
Batch 24 Loss: 0.3114
Batch 36 Loss: 0.2583

EPOCH 72 


EPOCH 127 LOSS: 0.2832

Epoch 128

Batch 0 Loss: 0.2622
Batch 12 Loss: 0.3709
Batch 24 Loss: 0.2661
Batch 36 Loss: 0.2879

EPOCH 128 LOSS: 0.2813

Epoch 129

Batch 0 Loss: 0.3328
Batch 12 Loss: 0.3055
Batch 24 Loss: 0.3007
Batch 36 Loss: 0.2552

EPOCH 129 LOSS: 0.2808

Epoch 130

Batch 0 Loss: 0.3005
Batch 12 Loss: 0.2562
Batch 24 Loss: 0.2717
Batch 36 Loss: 0.2995

EPOCH 130 LOSS: 0.2810

Epoch 131

Batch 0 Loss: 0.2458
Batch 12 Loss: 0.2608
Batch 24 Loss: 0.3031
Batch 36 Loss: 0.2745

EPOCH 131 LOSS: 0.2836

Epoch 132

Batch 0 Loss: 0.2453
Batch 12 Loss: 0.2700
Batch 24 Loss: 0.2346
Batch 36 Loss: 0.2673

EPOCH 132 LOSS: 0.2815

Epoch 133

Batch 0 Loss: 0.2807
Batch 12 Loss: 0.2625
Batch 24 Loss: 0.3953
Batch 36 Loss: 0.2606

EPOCH 133 LOSS: 0.2795

Epoch 134

Batch 0 Loss: 0.2952
Batch 12 Loss: 0.2906
Batch 24 Loss: 0.2638
Batch 36 Loss: 0.2969

EPOCH 134 LOSS: 0.2793

Epoch 135

Batch 0 Loss: 0.2559
Batch 12 Loss: 0.3323
Batch 24 Loss: 0.3316
Batch 36 Loss: 0.2899

EPOCH 135 LOSS:

KeyboardInterrupt: 

In [250]:
#Fine tune with smaller learning rate
optimizer = torch.optim.RMSprop(ae1.parameters(), lr=0.000001)
train_autoencoder(model=ae1,
                   dataset=normal_torch,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=512,
                   epochs=600,
                   MSE_stopping_threshold=0.05)


Epoch 1

Batch 0 Loss: 0.2398
Batch 5 Loss: 0.2749
Batch 10 Loss: 0.2869
Batch 15 Loss: 0.2683
Batch 20 Loss: 0.2434

EPOCH 1 LOSS: 0.2569


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/remycanario/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-250-8cb13fec846c>", line 9, in <module>
    MSE_stopping_threshold=0.05)
  File "<ipython-input-249-7cc726336f29>", line 51, in train_autoencoder
    if val==True:
UnboundLocalError: local variable 'val' referenced before assignment

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/remycanario/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'UnboundLocalError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/remycanario/anaconda3/lib/python3.7/site-packages/IPython/cor

UnboundLocalError: local variable 'val' referenced before assignment

#### Save results

In [92]:
output1 = ae1(normal_torch)
train_output1 = ae1(train_torch)
val_output1 = ae1(val_torch)

### Autoencoder II: Undercomplete

In [103]:
ae2 = AutoEncoder(n_features,int(n_features*0.8)//1)
loss_func = nn.MSELoss()

In [ ]:
optimizer = torch.optim.SGD(ae2.parameters(), lr=0.2,momentum=0.9,nesterov=True)
train_autoencoder(model=ae2,
                   dataset=output1,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=1024,
                   epochs=120,
                   MSE_stopping_threshold=0.05)

In [125]:
optimizer = torch.optim.SGD(ae2.parameters(), lr=0.05,momentum=0.9,nesterov=True)
train_autoencoder(model=ae2,
                   dataset=output1,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=1024,
                   epochs=120,
                   MSE_stopping_threshold=0.05)


Epoch 1

Batch 0 Loss: 0.0533
Batch 2 Loss: 0.0568
Batch 4 Loss: 0.0564
Batch 6 Loss: 0.0537
Batch 8 Loss: 0.0535
Batch 10 Loss: 0.0624

EPOCH 1 LOSS: 0.0554

Epoch 2

Batch 0 Loss: 0.0532
Batch 2 Loss: 0.0528
Batch 4 Loss: 0.0567
Batch 6 Loss: 0.0562
Batch 8 Loss: 0.0597
Batch 10 Loss: 0.0527

EPOCH 2 LOSS: 0.0553

Epoch 3

Batch 0 Loss: 0.0568
Batch 2 Loss: 0.0548
Batch 4 Loss: 0.0545
Batch 6 Loss: 0.0553
Batch 8 Loss: 0.0531
Batch 10 Loss: 0.0529

EPOCH 3 LOSS: 0.0553

Epoch 4

Batch 0 Loss: 0.0544
Batch 2 Loss: 0.0561
Batch 4 Loss: 0.0539
Batch 6 Loss: 0.0561
Batch 8 Loss: 0.0525
Batch 10 Loss: 0.0552

EPOCH 4 LOSS: 0.0552

Epoch 5

Batch 0 Loss: 0.0542
Batch 2 Loss: 0.0578
Batch 4 Loss: 0.0532
Batch 6 Loss: 0.0568
Batch 8 Loss: 0.0550
Batch 10 Loss: 0.0547

EPOCH 5 LOSS: 0.0551

Epoch 6

Batch 0 Loss: 0.0570
Batch 2 Loss: 0.0528
Batch 4 Loss: 0.0539
Batch 6 Loss: 0.0561
Batch 8 Loss: 0.0527
Batch 10 Loss: 0.0629

EPOCH 6 LOSS: 0.0550

Epoch 7

Batch 0 Loss: 0.0569
Batch 2 Loss: 0

Batch 8 Loss: 0.0527
Batch 10 Loss: 0.0541

EPOCH 49 LOSS: 0.0520

Epoch 50

Batch 0 Loss: 0.0567
Batch 2 Loss: 0.0509
Batch 4 Loss: 0.0494
Batch 6 Loss: 0.0539
Batch 8 Loss: 0.0541
Batch 10 Loss: 0.0508

EPOCH 50 LOSS: 0.0520

Epoch 51

Batch 0 Loss: 0.0514
Batch 2 Loss: 0.0507
Batch 4 Loss: 0.0511
Batch 6 Loss: 0.0542
Batch 8 Loss: 0.0507
Batch 10 Loss: 0.0516

EPOCH 51 LOSS: 0.0519

Epoch 52

Batch 0 Loss: 0.0518
Batch 2 Loss: 0.0526
Batch 4 Loss: 0.0505
Batch 6 Loss: 0.0506
Batch 8 Loss: 0.0494
Batch 10 Loss: 0.0519

EPOCH 52 LOSS: 0.0518

Epoch 53

Batch 0 Loss: 0.0531
Batch 2 Loss: 0.0519
Batch 4 Loss: 0.0498
Batch 6 Loss: 0.0493
Batch 8 Loss: 0.0537
Batch 10 Loss: 0.0527

EPOCH 53 LOSS: 0.0518

Epoch 54

Batch 0 Loss: 0.0522
Batch 2 Loss: 0.0528
Batch 4 Loss: 0.0513
Batch 6 Loss: 0.0497
Batch 8 Loss: 0.0550
Batch 10 Loss: 0.0500

EPOCH 54 LOSS: 0.0517

Epoch 55

Batch 0 Loss: 0.0508
Batch 2 Loss: 0.0503
Batch 4 Loss: 0.0536
Batch 6 Loss: 0.0511
Batch 8 Loss: 0.0506
Batch 10 Loss

#### Save results

In [127]:
output2 = ae2(output1)
train_output2 = ae2(train_output1)
val_output2 = ae2(val_output1)

### Autoencoder III: Denoising

In [134]:
ae3 = AutoEncoder(n_features,n_features)
loss_func = nn.MSELoss()

In [139]:
optimizer = torch.optim.SGD(ae3.parameters(), lr=0.2,momentum=0.9,nesterov=True)
train_autoencoder(model=ae3,
                  dataset=output2,
                  loss_func=loss_func,
                  optimizer=optimizer,
                  batch_size=512,
                  epochs=100,
                  noise_factor=0.25,
                  MSE_stopping_threshold=0.05)


Epoch 1

Batch 0 Loss: 0.4579
Batch 5 Loss: 0.4935
Batch 10 Loss: 0.4716
Batch 15 Loss: 0.5975
Batch 20 Loss: 0.5203

EPOCH 1 LOSS: 0.4308

Epoch 2

Batch 0 Loss: 0.5523
Batch 5 Loss: 0.3985
Batch 10 Loss: 0.4979
Batch 15 Loss: 0.5468
Batch 20 Loss: 0.5095

EPOCH 2 LOSS: 0.4220

Epoch 3

Batch 0 Loss: 0.5076
Batch 5 Loss: 0.4918
Batch 10 Loss: 0.4185
Batch 15 Loss: 0.4428
Batch 20 Loss: 0.4979

EPOCH 3 LOSS: 0.4135

Epoch 4

Batch 0 Loss: 0.4526
Batch 5 Loss: 0.4211
Batch 10 Loss: 0.4544
Batch 15 Loss: 0.4405
Batch 20 Loss: 0.4004

EPOCH 4 LOSS: 0.4054

Epoch 5

Batch 0 Loss: 0.4820
Batch 5 Loss: 0.4794
Batch 10 Loss: 0.4977
Batch 15 Loss: 0.4601
Batch 20 Loss: 0.4338

EPOCH 5 LOSS: 0.3969

Epoch 6

Batch 0 Loss: 0.4921
Batch 5 Loss: 0.3978
Batch 10 Loss: 0.4718
Batch 15 Loss: 0.4839
Batch 20 Loss: 0.4723

EPOCH 6 LOSS: 0.3886

Epoch 7

Batch 0 Loss: 0.4228
Batch 5 Loss: 0.4604
Batch 10 Loss: 0.4380
Batch 15 Loss: 0.4369
Batch 20 Loss: 0.4385

EPOCH 7 LOSS: 0.3800

Epoch 8

Batch 0 Lo


EPOCH 55 LOSS: 0.0962

Epoch 56

Batch 0 Loss: 0.1468
Batch 5 Loss: 0.1322
Batch 10 Loss: 0.1505
Batch 15 Loss: 0.1510
Batch 20 Loss: 0.1542

EPOCH 56 LOSS: 0.0939

Epoch 57

Batch 0 Loss: 0.1433
Batch 5 Loss: 0.1361
Batch 10 Loss: 0.1423
Batch 15 Loss: 0.1418
Batch 20 Loss: 0.1369

EPOCH 57 LOSS: 0.0911

Epoch 58

Batch 0 Loss: 0.1430
Batch 5 Loss: 0.1462
Batch 10 Loss: 0.1432
Batch 15 Loss: 0.1341
Batch 20 Loss: 0.1370

EPOCH 58 LOSS: 0.0886

Epoch 59

Batch 0 Loss: 0.1247
Batch 5 Loss: 0.1297
Batch 10 Loss: 0.1341
Batch 15 Loss: 0.1298
Batch 20 Loss: 0.1283

EPOCH 59 LOSS: 0.0863

Epoch 60

Batch 0 Loss: 0.1341
Batch 5 Loss: 0.1344
Batch 10 Loss: 0.1359
Batch 15 Loss: 0.1207
Batch 20 Loss: 0.1483

EPOCH 60 LOSS: 0.0849

Epoch 61

Batch 0 Loss: 0.1458
Batch 5 Loss: 0.1261
Batch 10 Loss: 0.1244
Batch 15 Loss: 0.1256
Batch 20 Loss: 0.1266

EPOCH 61 LOSS: 0.0820

Epoch 62

Batch 0 Loss: 0.1246
Batch 5 Loss: 0.1300
Batch 10 Loss: 0.1260
Batch 15 Loss: 0.1281
Batch 20 Loss: 0.1343

EPOCH

In [209]:
#get outputs
output3 = ae3(output2)
train_output3 = ae3(train_output2)
val_output3 = ae3(val_output2)

In [148]:
reconstruction_error = np.power(val_output3.detach().numpy() - X_val_sc, 2).sum(axis=1)
np.corrcoef(y_val,reconstruction_error)[0][1]

0.060780185124493354

### Autoencoder IV: L2 Regularization

In [184]:
ae4 = AutoEncoder(n_features,n_features)
loss_func = nn.MSELoss()

In [186]:
optimizer = torch.optim.RMSprop(ae4.parameters(), lr=0.05,weight_decay=0.1)
train_autoencoder(model=ae4,
                  dataset=output3,
                  loss_func=loss_func,
                  optimizer=optimizer,
                  batch_size=1024,
                  epochs=1000,
                  MSE_stopping_threshold=800)


Epoch 1

Batch 0 Loss: 111201.4609
Batch 2 Loss: 1277769728.0000
Batch 4 Loss: 11565046.0000
Batch 6 Loss: 8408589.0000
Batch 8 Loss: 4865637.5000
Batch 10 Loss: 14959231.0000

EPOCH 1 LOSS: 839132.9375

Epoch 2

Batch 0 Loss: 805263.5625
Batch 2 Loss: 1022034.1875
Batch 4 Loss: 1201974.6250
Batch 6 Loss: 2082187.5000
Batch 8 Loss: 322934.5938
Batch 10 Loss: 821120.1875

EPOCH 2 LOSS: 613904.9375

Change: -0.2684%

Epoch 3

Batch 0 Loss: 455966.0000
Batch 2 Loss: 1390130.8750
Batch 4 Loss: 178812.3594
Batch 6 Loss: 261915.1250
Batch 8 Loss: 87066.5312
Batch 10 Loss: 102809.1016

EPOCH 3 LOSS: 111145.0547

Change: -0.8190%

Epoch 4

Batch 0 Loss: 128099.8906
Batch 2 Loss: 73789.9062
Batch 4 Loss: 57657.7148
Batch 6 Loss: 69818.6875
Batch 8 Loss: 282149.6250
Batch 10 Loss: 172192.7031

EPOCH 4 LOSS: 85008.3047

Change: -0.2352%

Epoch 5

Batch 0 Loss: 45884.2578
Batch 2 Loss: 52504.9609
Batch 4 Loss: 61311.5820
Batch 6 Loss: 75955.2031
Batch 8 Loss: 373803.8125
Batch 10 Loss: 75845.6094

Batch 2 Loss: 6754.3325
Batch 4 Loss: 9284.8652
Batch 6 Loss: 5543.8760
Batch 8 Loss: 56834.8047
Batch 10 Loss: 46322.8164

EPOCH 39 LOSS: 37402.5352

Change: 2.0677%

Epoch 40

Batch 0 Loss: 21024.0898
Batch 2 Loss: 11973.8340
Batch 4 Loss: 16639.0410
Batch 6 Loss: 16491.6816
Batch 8 Loss: 5514.5469
Batch 10 Loss: 114080.0312

EPOCH 40 LOSS: 355778.0938

Change: 8.5121%

Epoch 41

Batch 0 Loss: 162301.4219
Batch 2 Loss: 670807.1250
Batch 4 Loss: 54793.1758
Batch 6 Loss: 42091.0156
Batch 8 Loss: 7599.4204
Batch 10 Loss: 6862.7407

EPOCH 41 LOSS: 23646.5488

Change: -0.9335%

Epoch 42

Batch 0 Loss: 8141.2124
Batch 2 Loss: 5233.7651
Batch 4 Loss: 133355.1875
Batch 6 Loss: 7358.1064
Batch 8 Loss: 13767.9648
Batch 10 Loss: 6009.1729

EPOCH 42 LOSS: 17327.6680

Change: -0.2672%

Epoch 43

Batch 0 Loss: 10530.3887
Batch 2 Loss: 7815.2788
Batch 4 Loss: 7157.4507
Batch 6 Loss: 101982.5156
Batch 8 Loss: 11875.1865
Batch 10 Loss: 4959.0781

EPOCH 43 LOSS: 10550.5947

Change: -0.3911%

Epoch 44


Batch 4 Loss: 2868.3228
Batch 6 Loss: 3151.8933
Batch 8 Loss: 4342.9175
Batch 10 Loss: 2648.0544

EPOCH 78 LOSS: 4177.3618

Change: 0.9535%

Epoch 79

Batch 0 Loss: 3523.1973
Batch 2 Loss: 1188.1519
Batch 4 Loss: 2870.8464
Batch 6 Loss: 1105.0439
Batch 8 Loss: 1175.5256
Batch 10 Loss: 3074.6169

EPOCH 79 LOSS: 1303.9476

Change: -0.6879%

Epoch 80

Batch 0 Loss: 993.5251
Batch 2 Loss: 2243.9099
Batch 4 Loss: 1040.6848
Batch 6 Loss: 952.5822
Batch 8 Loss: 1035.0676
Batch 10 Loss: 1263.1342

EPOCH 80 LOSS: 1145.1753

Change: -0.1218%

Epoch 81

Batch 0 Loss: 1068.1921
Batch 2 Loss: 899.6949
Batch 4 Loss: 931.6595
Batch 6 Loss: 968.2451
Batch 8 Loss: 2094.3518
Batch 10 Loss: 1312.2133

EPOCH 81 LOSS: 1026.7164

Change: -0.1034%

Epoch 82

Batch 0 Loss: 1700.1667
Batch 2 Loss: 1247.7678
Batch 4 Loss: 832.0862
Batch 6 Loss: 841.0358
Batch 8 Loss: 1995.7563
Batch 10 Loss: 956.9216

EPOCH 82 LOSS: 990.0681

Change: -0.0357%

Epoch 83

Batch 0 Loss: 1179.0166
Batch 2 Loss: 777.6226
Batch 4 Los

In [259]:
optimizer = torch.optim.RMSprop(ae4.parameters(), lr=0.00005,weight_decay=0.1)
train_autoencoder(model=ae4,
                  dataset=output3,
                  loss_func=loss_func,
                  optimizer=optimizer,
                  validation_tensor=val_torch,
                  batch_size=1024,
                  epochs=1000,
                  MSE_stopping_threshold=0.5)


Epoch 1

Batch 0 Loss: 0.5414
Batch 2 Loss: 0.5210
Batch 4 Loss: 0.5100
Batch 6 Loss: 0.5565
Batch 8 Loss: 0.5580
Batch 10 Loss: 0.6355

EPOCH 1 LOSS: 0.5447

Reconstruction error/fraud corr: 0.0613

Epoch 2

Batch 0 Loss: 0.6362
Batch 2 Loss: 0.5360
Batch 4 Loss: 0.4963
Batch 6 Loss: 0.5047
Batch 8 Loss: 0.6408
Batch 10 Loss: 0.5216

EPOCH 2 LOSS: 0.5444

Change: -0.0005%

Reconstruction error/fraud corr: 0.0606

Epoch 3

Batch 0 Loss: 0.5510
Batch 2 Loss: 0.5366
Batch 4 Loss: 0.5867
Batch 6 Loss: 0.5172
Batch 8 Loss: 0.5622
Batch 10 Loss: 0.5313

EPOCH 3 LOSS: 0.5441

Change: -0.0006%

Reconstruction error/fraud corr: 0.0615

Epoch 4

Batch 0 Loss: 0.5228
Batch 2 Loss: 0.5429
Batch 4 Loss: 0.5092
Batch 6 Loss: 0.4958
Batch 8 Loss: 0.5580
Batch 10 Loss: 0.7144

EPOCH 4 LOSS: 0.5440

Change: -0.0002%

Reconstruction error/fraud corr: 0.0630

Epoch 5

Batch 0 Loss: 0.5870
Batch 2 Loss: 0.5163
Batch 4 Loss: 0.5442
Batch 6 Loss: 0.5865
Batch 8 Loss: 0.5609
Batch 10 Loss: 0.5523

EPOCH 5 

KeyboardInterrupt: 

In [261]:
#get outputs
output4 = ae4(output3)
train_output4 = ae4(train_output3)
val_output4 = ae4(val_output3)

## Autoencoder V: L1 Regularization

In [315]:
ae5 = AutoEncoder(n_features,n_features)

def L1_loss(recon,inputs):
    MSELoss = nn.MSELoss()
    loss = MSELoss(recon,inputs)
    for param in ae5.parameters():
        loss += lmbda*torch.sum(torch.abs(param))
    return loss

loss_func = L1_loss

In [325]:
lmbda=0.4
optimizer = torch.optim.SGD(ae5.parameters(), lr=1e-8,momentum=0.9,nesterov=True)
train_autoencoder(model=ae5,
                   dataset=output4,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=1024,
                   epochs=120,
                   validation_tensor=val_torch,
                   MSE_stopping_threshold=0.05)


Epoch 1

Batch 0 Loss: 0.1822
Batch 2 Loss: 0.1249
Batch 4 Loss: 0.0621
Batch 6 Loss: 0.0462
Batch 8 Loss: 0.0548
Batch 10 Loss: 0.0448

EPOCH 1 LOSS: 0.0313

Reconstruction error/fraud corr: 0.0937


In [326]:
output5 = ae5(output4)
train_output5 = ae5(train_output4)
val_output5 = ae5(val_output4)

In [ ]:
final_train_output = train_output5.detach().numpy()
final_val_output = train_output5.detach().numpy()
train_reconstruction_score = sc.fit_transform(np.power(X_train_sc - final_train_output,2).sum(axis=1))
val_reconstruction_score = sc.fit_transform(np.power(X_val_sc - final_val_output,2).sum(axis=1))

# Penalized Maximum Likelihood Estimation
Using Firth's Logistic Regression with Intercept Correction

In [ ]:
FLIC = PMLE.Firth_Logit(FLIC=True)
FLIC.fit(X_train_sc,y_train)
FLIC_train_preds = FLIC.predict_proba(X_train_sc)
FLIC_val_preds = FLIC.predict_proba(X_train_sc)

# SMOTE Dataset Resampling
Estimation with XGBoost

## Perform Synthetic Minority Over-Sampling Technique

In [ ]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train_sc,y_train)
X_val_smote, y_val_smote = smote.fit_resample(X_val_sc,y_val)

## Get XGBoost estimate of SMOTE dataset

In [ ]:
#Instantiate XGBoost and grid search
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb_gs = GridSearchCV(xgb,params,cv=3)

#fit model and obtain predictions
xgb_gs.fit(X_train_smote,y_train_smote)

smote_train_preds = xgb_gs.predict_proba(X_train_smote)
smote_val_preds = xgb_gs.predict_proba(X_val_smote)

# Ensemble results

## Create Ensemble DFs

In [ ]:
ensemble_X_train = pd.DataFrame(None)
ensemble_X_train['anomaly_score'] = train_reconstruction_score
ensemble_X_train['FLIC'] = FLIC_train_preds
ensemble_X_train['smote'] = smote_train_preds

ensemble_X_val = pd.DataFrame(None)
ensemble_X_val['anomaly_score'] = val_reconstruction_score
ensemble_X_val['FLIC'] = FLIC_val_preds
ensemble_X_val['smote'] = smote_val_preds

## Get results using FLIC 

In [ ]:
FLIC.fit(ensemble_X_train,y_train)
preds = FLIC.predict_proba(ensemble_X_val)